In [69]:
import pandas as pd  
import datetime
import os
from Historic_Crypto import HistoricalData
from binance.spot import Spot
from dotenv import load_dotenv

load_dotenv()
def get_past_date(years=3):
    """Returns a date 3 years in the past in the format "yyyy-mm-dd-hh-mm".

    Args:
        years (int, optional): Number of years in the past. Defaults to 3.

    Returns:
        str: Date in the format "yyyy-mm-dd-hh-mm".
    """

    current_date = datetime.datetime.utcnow()
    past_date = current_date - datetime.timedelta(days=365 * years)
    return past_date.strftime("%Y-%m-%d-%H-%M")

def now_date():
    """Returns the current date and time in the format "yyyy-mm-dd-hh-mm"."""

    current_time_utc = datetime.datetime.utcnow() - datetime.timedelta(days=1)
    return current_time_utc.strftime("%Y-%m-%d-%H-%M")

def save_to_csv(df, filename, overwrite=True):
    """
    Saves a Pandas DataFrame to a CSV file with overwrite option.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        filename (str): The desired filename (including path).
        overwrite (bool, optional): Whether to overwrite existing file (default: True).
    """

    # Create data directory if it doesn't exist
    data_dir = os.path.join(os.getcwd(), "data")
    os.makedirs(data_dir, exist_ok=True)

    # Check for existing file and overwrite if allowed
    full_path = os.path.join(data_dir, filename + '.csv')
    if os.path.exists(full_path) and not overwrite:
        raise FileExistsError(f"File already exists: {full_path}")

    # Save DataFrame to CSV
    df.to_csv(full_path, index=False)

    print(f"DataFrame saved to: {full_path}")
    return full_path

def get_crypto_prices_coinbase(ticker:str, start_date:str, end_date:str, granularity:int=86400):
    """Retrieves crypto prices from API, handling date range splits and data frame appending.

    Args:
        ticker (str): a Cryptocurrency ticker
        start_date (str): Start date in "yyyy-mm-dd-hh-mm" format.
        end_date (str): End date in "yyyy-mm-dd-hh-mm" format.
        granularity (str): price data frequency in second.

    Returns:
        pd.DataFrame: Combined DataFrame with all retrieved prices.
    """

    start_datetime = pd.to_datetime(start_date, format="%Y-%m-%d-%H-%M")
    end_datetime = pd.to_datetime(end_date, format="%Y-%m-%d-%H-%M")
    
    all_data = pd.DataFrame()
    current_date = start_datetime
    

    while current_date <= end_datetime:
        next_date = min(current_date + pd.Timedelta(days=300), end_datetime)  # Limit to 300 days
        dates = pd.date_range(start=current_date, end=next_date - pd.Timedelta(seconds=1), freq=str(granularity) + 'S')
        
        try:
            current_range_data = HistoricalData(ticker, granularity, current_date.strftime("%Y-%m-%d-%H-%M"), next_date.strftime("%Y-%m-%d-%H-%M")).retrieve_data()
            print(current_range_data)
            
            if pd.__version__ >= '1.0.0':
                current_range_data.insert(0, 'Date', dates.to_numpy().astype('datetime64[D]').astype('str'))  # For Pandas >= 1.0.0
            else:
                current_range_data.insert(0, 'Date', dates.date)  # For Pandas < 1.0.0
        except:
            # Create empty DataFrame with expected columns and dates for missing data
            current_range_data = pd.DataFrame(index=dates, columns=["Date", "time", "low", "high", "open", "close", "volume"])
            current_range_data['Date'] = dates.to_numpy().astype('datetime64[D]').astype('str')  # Fill Date column
            current_range_data.fillna(method='ffill', inplace=True)  # Forward-fill missing values
            
        all_data = pd.concat([all_data, current_range_data],ignore_index=True)
        current_date = next_date + pd.Timedelta(days=1)  # Move to next day, 1 minute after the last call

    csv_data = save_to_csv(all_data,ticker)
    return csv_data


In [71]:
client = Spot()
print(client.time())

client = Spot(key=os.getenv('BINANCE_KEY'), secret=os.getenv('BINANCE_SECRET'))


{'serverTime': 1704077179309}


TypeError: API.__init__() got an unexpected keyword argument 'key'

In [67]:
ticker = "ETH-USD"

csv_data = get_crypto_prices_coinbase(ticker, get_past_date(), now_date())

Checking input parameters are in the correct format.
Formatting Dates.
Checking if user supplied is available on the CoinBase Pro API.
Connected to the CoinBase Pro API.
Ticker 'ETH-USD' found at the CoinBase Pro API, continuing to extraction.
Retrieved Data from Coinbase Pro API.
Returning data.
                low     high     open    close        volume
time                                                        
2021-01-01   717.10   750.00   737.89   730.99  1.980604e+05
2021-01-02   715.22   789.21   730.97   775.30  3.688993e+05
2021-01-03   770.00  1015.14   775.29   980.44  8.587438e+05
2021-01-04   886.33  1168.99   980.44  1042.38  1.103883e+06
2021-01-05   975.00  1136.93  1042.24  1103.73  8.584959e+05
...             ...      ...      ...      ...           ...
2021-10-23  3935.92  4172.04  3971.89  4168.56  1.092536e+05
2021-10-24  3962.05  4188.42  4168.53  4082.64  1.267315e+05
2021-10-25  4069.22  4236.27  4082.09  4220.93  1.457538e+05
2021-10-26  4091.31  4297.00  4